In [3]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [4]:
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance

In [5]:
vector_db = QdrantClient(host='localhost', port=6333)
collection_name = 'specs_collection'
vector_dim = embedding_model.get_sentence_embedding_dimension()
vector_dim

if not vector_db.collection_exists(collection_name):
    vector_db.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=vector_dim,
            distance=Distance.COSINE
        )
    )

In [7]:
TOP_K = 3
question = "how much is the power of PC200-10M0?"

retrieved_chunks = vector_db.query_points(
    collection_name=collection_name,
    query=embedding_model.encode(question),
    limit=TOP_K
)

In [8]:
retrieved_chunks.points

[ScoredPoint(id='3719502a-fa83-4b28-98cb-b137b0b01aef', version=0, score=0.4325035, payload={'text': 'Notes - "ISO 6015" - "PC200-10MO" logo and illustration with machines Data and details are taken directly from the document provided.Here is the extracted data from the document: ### General Information - **Model**: PC200-10MO - **Arm Length**: 2925 mm - **Conditions**: 5700 mm one-piece boom ### Key - **A**: Reach from swing center - **B**: Arm top pin height - **C**: Lifting capacity - **Cf**: Rating over front - **Cs**: Rating over side - **⚫**: Rating at maximum reach ### Table: PC200-10MO Arm: 2925 mm Without bucket, Shoe: 600 mm triple grouser | B | A 7.5m | 6.0m | 4.5m | 3.0m | 1.5m | |---------|---------|-------|-------|-------|-------| | MAX | 5.68m | 6120 kg | 3930 kg | | | | | 7.5m | 6.42m | Cf 3340 kg | Cs 3340 kg | | 6.0m | 7.49m | Cf 3190 kg | Cs 3080 kg | Cf 3560 kg | Cs 3090 kg | Cf 4750 kg | Cs 4560 kg | | 5.5m | 8.15m | Cf 3190 kg | Cs 2690 kg | Cf 4470 kg | Cs 3030 k

In [9]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
context = '\n'.join(["- " + chunk.payload['text'] for chunk in retrieved_chunks.points])
print(context)

- Notes - "ISO 6015" - "PC200-10MO" logo and illustration with machines Data and details are taken directly from the document provided.Here is the extracted data from the document: ### General Information - **Model**: PC200-10MO - **Arm Length**: 2925 mm - **Conditions**: 5700 mm one-piece boom ### Key - **A**: Reach from swing center - **B**: Arm top pin height - **C**: Lifting capacity - **Cf**: Rating over front - **Cs**: Rating over side - **⚫**: Rating at maximum reach ### Table: PC200-10MO Arm: 2925 mm Without bucket, Shoe: 600 mm triple grouser | B | A 7.5m | 6.0m | 4.5m | 3.0m | 1.5m | |---------|---------|-------|-------|-------|-------| | MAX | 5.68m | 6120 kg | 3930 kg | | | | | 7.5m | 6.42m | Cf 3340 kg | Cs 3340 kg | | 6.0m | 7.49m | Cf 3190 kg | Cs 3080 kg | Cf 3560 kg | Cs 3090 kg | Cf 4750 kg | Cs 4560 kg | | 5.5m | 8.15m | Cf 3190 kg | Cs 2690 kg | Cf 4470 kg | Cs 3030 kg | Cf 5390 kg | Cs 4370 kg | Cf 6010 kg | Cs 6010 kg | | 4.5m | 8.15m | Cf 3340 kg || Cf 4340 kg | 

In [ ]:
llm = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

response = llm.chat.completions.create(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    messages=[
        {"role": "system", "content": "context: \n" + context},
        {"role": "user", "content": question}
    ]
)

In [15]:
print(response.choices[0].message.content)

The power of PC200-10M0 is:

* Gross: 110 kW / 148 HP at 2000 rpm
* Net: 103 kW / 138 HP at 2000 rpm
